In [1]:
import nest_asyncio

nest_asyncio.apply()

from llama_index.core import SimpleDirectoryReader

# Load Data

# reader = SimpleDirectoryReader("./ingestions/output_files/")
reader = SimpleDirectoryReader(input_files=["./notebooks/PMC10003329.txt"])
documents = reader.load_data()

documents[0]

Document(id_='fe11dfc2-8461-4acf-8ae1-cf583af2f8f0', embedding=None, metadata={'file_path': 'notebooks/PMC10003329.txt', 'file_name': 'PMC10003329.txt', 'file_type': 'text/plain', 'file_size': 42380, 'creation_date': '2024-08-12', 'last_modified_date': '2024-08-12'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='\n==== Front\nJ Clin Med\nJ Clin Med\njcm\nJournal of Clinical Medicine\n2077-0383\nMDPI\n\n10.3390/jcm12051716\njcm-12-01716\nArticle\nVenous Thrombus Embolism in Polytrauma: Special Attention to Patients with Traumatic Brain Injury\nChen Deng 1\nhttps://orcid.org/0000-0001-8179-5236\nLuo Jialiu 1\nZhang Cong 1\nTang Liangsheng 1\nhttps://orcid.org/0000-0002-2337-5222\nDeng Hai 1\nChang Teding 1\nXu Huaqiang 2\nHe Miaobo 2\nWan Dongli 2\nZhang

In [3]:
from semantic_router.encoders import OpenAIEncoder
from llama_index.embeddings.openai import OpenAIEmbedding, OpenAIEmbeddingModelType

encoder = OpenAIEncoder(name=OpenAIEmbeddingModelType.TEXT_EMBED_3_SMALL)

/Users/holeminhthach/Library/Caches/pypoetry/virtualenvs/clinfo-ai-CU3KyI9c-py3.12/lib/python3.12/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [4]:
from semantic_chunkers import StatisticalChunker

docs = [str(doc.get_text()) for doc in documents]

chunker = StatisticalChunker(encoder=encoder)
chunks = chunker(docs=docs)

2024-08-12 21:36:52 INFO semantic_chunkers.utils.logger Single document exceeds the maximum token limit of 300. Splitting to sentences before semantically merging.


  0%|          | 0/15 [00:00<?, ?it/s]

In [5]:
chunker.print(chunks[0])

Split 1, tokens 202, triggered by: 0.26
==== Front J Clin Med J Clin Med jcm Journal of Clinical Medicine 2077-0383 MDPI 10.3390/jcm12051716 jcm-12-01716 Article Venous Thrombus Embolism in Polytrauma: Special Attention to Patients with Traumatic Brain Injury Chen Deng 1 https://orcid.org/0000-0001-8179-5236 Luo Jialiu 1 Zhang Cong 1 Tang Liangsheng 1 https://orcid.org/0000-0002-2337-5222 Deng Hai 1 Chang Teding 1 Xu Huaqiang 2 He Miaobo 2 Wan Dongli 2 Zhang Feiyu 3 Wu Mengfan 3 Qian Min 4 Zhou Wen 4 Yin Gang 3* Wang Wenguo 2* Dong Liming 1* Tang Zhaohui 1*
----------------------------------------------------------------------------------------


Split 2, tokens 209, triggered by: 0.19
Van der Naalt Joukje Academic Editor Jacobs Bram Academic Editor 1 Department of Trauma Surgery, Tongji Trauma Center, Tongji Hospital, Tongji Medical College, Huazhong University of Science and Technology, Wuhan 430030, China 2 Intensive Care Unit, Trauma Center, Suizhou Central Hospital, Hubei Universi

In [7]:
chunker.print(chunks[1])

IndexError: list index out of range

In [8]:
with open("./notebooks/PMC10003329.txt") as file:
    essay = file.read()

In [9]:
import re

# Splitting the essay on '.', '?', and '!'
single_sentences_list = re.split(r"(?<=[.?!])\s+", essay)
print(f"{len(single_sentences_list)} senteneces were found")

492 senteneces were found


In [10]:
sentences = [{"sentence": x, "index": i} for i, x in enumerate(single_sentences_list)]
sentences[:3]

[{'sentence': '\n==== Front\nJ Clin Med\nJ Clin Med\njcm\nJournal of Clinical Medicine\n2077-0383\nMDPI\n\n10.3390/jcm12051716\njcm-12-01716\nArticle\nVenous Thrombus Embolism in Polytrauma: Special Attention to Patients with Traumatic Brain Injury\nChen Deng 1\nhttps://orcid.org/0000-0001-8179-5236\nLuo Jialiu 1\nZhang Cong 1\nTang Liangsheng 1\nhttps://orcid.org/0000-0002-2337-5222\nDeng Hai 1\nChang Teding 1\nXu Huaqiang 2\nHe Miaobo 2\nWan Dongli 2\nZhang Feiyu 3\nWu Mengfan 3\nQian Min 4\nZhou Wen 4\nYin Gang 3*\nWang Wenguo 2*\nDong Liming 1*\nTang Zhaohui 1*\nVan der Naalt Joukje Academic Editor\nJacobs Bram Academic Editor\n1 Department of Trauma Surgery, Tongji Trauma Center, Tongji Hospital, Tongji Medical College, Huazhong University of Science and Technology, Wuhan 430030, China\n2 Intensive Care Unit, Trauma Center, Suizhou Central Hospital, Hubei University of Medicine, Suizhou 441300, China\n3 Department of Trauma Surgery, Trauma Center, Tianmen First People’s Hospital, 

In [11]:
def combine_sentences(sentences, buffer_size=1):
    # Go through each sentence dict
    for i in range(len(sentences)):

        # Create a string that will hold the sentences which are joined
        combined_sentence = ""

        # Add sentences before the current one, based on the buffer size.
        for j in range(i - buffer_size, i):
            # Check if the index j is not negative (to avoid index out of range like on the first one)
            if j >= 0:
                # Add the sentence at index j to the combined_sentence string
                combined_sentence += sentences[j]["sentence"] + " "

        # Add the current sentence
        combined_sentence += sentences[i]["sentence"]

        # Add sentences after the current one, based on the buffer size
        for j in range(i + 1, i + 1 + buffer_size):
            # Check if the index j is within the range of the sentences list
            if j < len(sentences):
                # Add the sentence at index j to the combined_sentence string
                combined_sentence += " " + sentences[j]["sentence"]

        # Then add the whole thing to your dict
        # Store the combined sentence in the current sentence dict
        sentences[i]["combined_sentence"] = combined_sentence

    return sentences


sentences = combine_sentences(sentences)

In [12]:
sentences[:3]

[{'sentence': '\n==== Front\nJ Clin Med\nJ Clin Med\njcm\nJournal of Clinical Medicine\n2077-0383\nMDPI\n\n10.3390/jcm12051716\njcm-12-01716\nArticle\nVenous Thrombus Embolism in Polytrauma: Special Attention to Patients with Traumatic Brain Injury\nChen Deng 1\nhttps://orcid.org/0000-0001-8179-5236\nLuo Jialiu 1\nZhang Cong 1\nTang Liangsheng 1\nhttps://orcid.org/0000-0002-2337-5222\nDeng Hai 1\nChang Teding 1\nXu Huaqiang 2\nHe Miaobo 2\nWan Dongli 2\nZhang Feiyu 3\nWu Mengfan 3\nQian Min 4\nZhou Wen 4\nYin Gang 3*\nWang Wenguo 2*\nDong Liming 1*\nTang Zhaohui 1*\nVan der Naalt Joukje Academic Editor\nJacobs Bram Academic Editor\n1 Department of Trauma Surgery, Tongji Trauma Center, Tongji Hospital, Tongji Medical College, Huazhong University of Science and Technology, Wuhan 430030, China\n2 Intensive Care Unit, Trauma Center, Suizhou Central Hospital, Hubei University of Medicine, Suizhou 441300, China\n3 Department of Trauma Surgery, Trauma Center, Tianmen First People’s Hospital, 

In [13]:
from langchain.embeddings import OpenAIEmbeddings

oaiembeds = OpenAIEmbeddings(model=OpenAIEmbeddingModelType.TEXT_EMBED_3_SMALL)

/Users/holeminhthach/Library/Caches/pypoetry/virtualenvs/clinfo-ai-CU3KyI9c-py3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [27]:
embeddings = oaiembeds.embed_documents([x["combined_sentence"] for x in sentences])